## This tutorial code is sourced from the [official documentation](https://pymongo.readthedocs.io/en/stable/examples/index.html).

In [1]:
!python -c "import requests; print(requests.get('https://www.howsmyssl.com/a/check', verify=False).json()['tls_version'])"

TLS 1.3


C:\Users\kyeul\anaconda3\envs\env-pymongo\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.howsmyssl.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [2]:
from pymongo import MongoClient
from dotenv import dotenv_values

In [3]:
# read env file
config = dotenv_values(".env") # .env 파일에서 환경변수 가져오기

In [4]:
mongodb_client = MongoClient(config['ATLAS_URI'])

In [5]:
db = mongodb_client.test_database

In [6]:
collection = db.test_collection

# Documents

In [7]:
import datetime
from pytz import timezone

In [16]:
KST = timezone("Asia/Seoul")
datetime.datetime.now(tz=KST)

datetime.datetime(2024, 1, 3, 9, 39, 56, 743216, tzinfo=<DstTzInfo 'Asia/Seoul' KST+9:00:00 STD>)

In [9]:
post = {
    'author' : 'Mike',
    'text' : 'My first blog post!',
    'tags': ['mongodb', 'python', 'pymongo'], # list 형태도 들어가고
    'date' : datetime.datetime.now(tz=KST),
}

# Inserting a Document

In [12]:
posts = db.posts

In [13]:
# we can use the 'insert_one()' method
post_id = posts.insert_one(post).inserted_id
post_id

ObjectId('6594ab8db88780a7d843435e')

In [14]:
db.list_collection_names()

['posts']

# Getting a Single Document With __find_one()__

In [15]:
# 하나의 document에 접근하거나 여러개 중 첫번째 document에 접근하려면 find_one()을 사용하면 된다.
import pprint
pprint.pprint(posts.find_one())

{'_id': ObjectId('6594ab8db88780a7d843435e'),
 'author': 'Mike',
 'date': datetime.datetime(2024, 1, 3, 0, 34, 11, 822000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


In [19]:
# 이렇게 특정 key-value로도 검색할 수 있다.
posts.find_one({"author": "Mike"})

{'_id': ObjectId('6594ab8db88780a7d843435e'),
 'author': 'Mike',
 'text': 'My first blog post!',
 'tags': ['mongodb', 'python', 'pymongo'],
 'date': datetime.datetime(2024, 1, 3, 0, 34, 11, 822000)}

In [20]:
# 당연히 id로도 할 수 있다.
post_id

ObjectId('6594ab8db88780a7d843435e')

In [21]:
posts.find_one({"_id": post_id})

{'_id': ObjectId('6594ab8db88780a7d843435e'),
 'author': 'Mike',
 'text': 'My first blog post!',
 'tags': ['mongodb', 'python', 'pymongo'],
 'date': datetime.datetime(2024, 1, 3, 0, 34, 11, 822000)}

In [22]:
# 주의할 것은 str로 하면 안된다는 것!
post_id_as_str = str(post_id)
posts.find_one({"_id": post_id_as_str}) # No result

# Bulk Insert

In [25]:
# insert_many()를 사용해서 list 타입으로 데이터를 넘기면 여러 데이터를 한번에 삽입할 수 있다.
new_posts = [
    {
        'author': 'Mike',
        'text': 'Another post!',
        'tags' : ['bulk', 'insert'],
        'date' : datetime.datetime(2024, 1, 3, 10, 52),
    },
    {
        'author' : 'Eliot',
        'title' : 'MongoDB is fun',
        'text' : 'and pretty easy too!',
        'date' : datetime.datetime(2024, 1, 3, 10, 53),
    }
]



In [26]:
result = posts.insert_many(new_posts)
result.inserted_ids

[ObjectId('6594be50b88780a7d843435f'), ObjectId('6594be50b88780a7d8434360')]

# Querying for More Than One Document

To get more than a single document as the result of a query we use the `find()` method. `find()` returns a `Cursor` instance, which allows us to iterate over all matching documents. For example, we can iterate over every document in the `posts` collection:

In [28]:
for post in posts.find(): # iterable 값이기 때문에 for 문에서 하나씩 접근 가능
    pprint.pprint(post)

{'_id': ObjectId('6594ab8db88780a7d843435e'),
 'author': 'Mike',
 'date': datetime.datetime(2024, 1, 3, 0, 34, 11, 822000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}
{'_id': ObjectId('6594be50b88780a7d843435f'),
 'author': 'Mike',
 'date': datetime.datetime(2024, 1, 3, 10, 52),
 'tags': ['bulk', 'insert'],
 'text': 'Another post!'}
{'_id': ObjectId('6594be50b88780a7d8434360'),
 'author': 'Eliot',
 'date': datetime.datetime(2024, 1, 3, 10, 53),
 'text': 'and pretty easy too!',
 'title': 'MongoDB is fun'}


Just like we did with `find_one()`, we can pass a document to `find()` to limit the returned results. Here, we get only those documents whose author is "Mike":

In [29]:
for post in posts.find({"author":"Mike"}):
    pprint.pprint(post)

{'_id': ObjectId('6594ab8db88780a7d843435e'),
 'author': 'Mike',
 'date': datetime.datetime(2024, 1, 3, 0, 34, 11, 822000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}
{'_id': ObjectId('6594be50b88780a7d843435f'),
 'author': 'Mike',
 'date': datetime.datetime(2024, 1, 3, 10, 52),
 'tags': ['bulk', 'insert'],
 'text': 'Another post!'}


# Counting

If we just want to know how many documents match a query, we can perform a `count_documents()` operation instead of a full query. We can get a count of all of the documents in collection:

In [30]:
posts.count_documents({})

3

or just of those documents that match a specific query:

In [31]:
posts.count_documents({'author':'Mike'})

2

# Range Queries

MongoDB supports many different types of advanced queries. As an example, lets perform a query where we limit results to posts older than a certain date, but also sort the results by author:

In [39]:
d = datetime.datetime(2024, 1, 10, 9)
for post in posts.find({
    "date": {"$lt": d}
}).sort("author"):
    pprint.pprint(post)


{'_id': ObjectId('6594be50b88780a7d8434360'),
 'author': 'Eliot',
 'date': datetime.datetime(2024, 1, 3, 10, 53),
 'text': 'and pretty easy too!',
 'title': 'MongoDB is fun'}
{'_id': ObjectId('6594ab8db88780a7d843435e'),
 'author': 'Mike',
 'date': datetime.datetime(2024, 1, 3, 0, 34, 11, 822000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}
{'_id': ObjectId('6594be50b88780a7d843435f'),
 'author': 'Mike',
 'date': datetime.datetime(2024, 1, 3, 10, 52),
 'tags': ['bulk', 'insert'],
 'text': 'Another post!'}


Here we use the special `"$lt"` operator to do a range query, and also call `sort()` to sort the results by author.

# Indexing

In [41]:
import pymongo

In [42]:
# First, we'll need to create the index:
result = db.profiles.create_index([("user_id", pymongo.ASCENDING)], unique=True)
sorted(list(db.profiles.index_information()))

['_id_', 'user_id_1']

Notice that we have two indices now: one is the index on `_id` that MongoDB creates automatically, and the other is the index on `user_id` we just created.

Now let's set up some user profiles:

In [43]:
user_profiles = [{"user_id": 211, "name" :"Luke"}, {"user_id": 212, "name": "ziltoid"}]
result = db.profiles.insert_many(user_profiles)

In [46]:
for i in db.profiles.find():
    pprint.pprint(i)

{'_id': ObjectId('6594c442b88780a7d8434361'), 'name': 'Luke', 'user_id': 211}
{'_id': ObjectId('6594c442b88780a7d8434362'), 'name': 'ziltoid', 'user_id': 212}


The index prevents us from inserting a document whose `user_id` is already in the collection:

In [49]:
new_profile = {"user_id": 213, "name" : "Drew"}
duplicate_profile = {"user_id":212, "name":"Tommy"}
result = db.profiles.insert_one(new_profile) # This is fine.

In [50]:
# This occurs error.
result = db.profiles.insert_one(duplicate_profile)

DuplicateKeyError: E11000 duplicate key error collection: test_database.profiles index: user_id_1 dup key: { user_id: 212 }, full error: {'index': 0, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: test_database.profiles index: user_id_1 dup key: { user_id: 212 }', 'keyPattern': {'user_id': 1}, 'keyValue': {'user_id': 212}}